In [ ]:
import torch
import torchvision
import os
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
import torch.nn as nn
from torch.optim import Adam
import numpy as np
from tqdm import tqdm

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6


# Data Loading

### Download and unzip the dataset

In [ ]:
!gdown https://drive.google.com/uc?id=1BqkjrY4VzghDFKlMGE-gafrqCvmVlm3f

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1BqkjrY4VzghDFKlMGE-gafrqCvmVlm3f 



In [ ]:
!unzip /content/scenery.zip;

unzip:  cannot find or open /content/scenery.zip, /content/scenery.zip.zip or /content/scenery.zip.ZIP.


### Loading and dividing the dataset into train, val and test sets

Initializing ImageFolder Instance

In [ ]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Resize((90,90))]) #resize images and load into tensor

data_path = '/content/scenery/'

dataset = torchvision.datasets.ImageFolder(root=data_path, transform=transform) #transform/load data, indexes and loads up the set of sub-dirs

FileNotFoundError: ignored

Attributes of the dataset object

In [ ]:
len(dataset) # number of samples

In [ ]:
dataset.classes # classes(build in?????, figure this out later)

In [ ]:
dataset.class_to_idx # indices of corresponding labels

In [ ]:
dataset.transform

In [ ]:
class_counts = np.zeros(len(dataset.class_to_idx))

In [ ]:
for image, label in dataset:
    class_counts[label] += 1

In [ ]:
for i in range(len(class_counts)):
  print("class:%s, instances: %d"%([k for k,v in dataset.class_to_idx.items() if v == i], class_counts[i]))

In [ ]:
def display_img(img,label):
    #print(f"Label : {dataset.classes[label]}")
    plt.figure()
    plt.title(f"Label : {dataset.classes[label]}")
    plt.imshow(img.permute(1,2,0))

#display the first image in the dataset
for i in range(1,4):
  display_img(*dataset[i])

Split into train, validation and test sets

In [ ]:
# test train split, utils package, hard coded??? takes fractions??? check later,,,,, manual seed >> random seeding!!!
train_set, val_set, test_set = torch.utils.data.random_split(dataset, [11924, 2555, 2555], generator=torch.Generator().manual_seed(42))

In [ ]:
train_set

In [ ]:
train_set.indices

In [ ]:
train_set.dataset # dataset means train set is a part/subset of the dataset!,

In [ ]:
def display_img(img,label):
    print(f"Label : {dataset.classes[label]}")
    plt.imshow(img.permute(1,2,0))

#display the first image in the dataset
display_img(*train_set[100])

Initializing the pytorch dataloaders

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=16,
    shuffle=True
)

val_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=16,
    shuffle=False
)

test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=16,
    shuffle=False
)


In [ ]:
from torchvision.utils import make_grid

def show_batch(loader):
    """Plot images grid of single batch"""
    for images, labels in loader:
        fig,ax = plt.subplots(figsize = (16,12))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(make_grid(images,nrow=16).permute(1,2,0))
        break

show_batch(val_loader)

# Convolutional Neural Networks

## nn.Module method of constructing models

In [ ]:
#TF>> sequential - simple
#  >> modular!! >> complex(flexible)



#Modular way!!!!
#torch.set_default_tensor_type('torch.cuda.FloatTensor')

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__() #init parent class

        # Defining a 2D convolution layer
        self.conv1 = Conv2d(3, 4, kernel_size=3, stride=1, padding=1)#(input channels, output channels,------)
        self.bn1 = BatchNorm2d(4)
        self.relu1 = ReLU(inplace=True) ## note thr inplace, is it important????
        self.maxpool1 = MaxPool2d(kernel_size=2, stride=2)

        # Defining another 2D convolution layer
        self.conv2 = Conv2d(4, 8, kernel_size=3, stride=1, padding=1)
        self.bn2 = BatchNorm2d(8)
        self.relu2 = ReLU(inplace=True)
        self.maxpool2 = MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = Conv2d(8, 16, kernel_size=3, stride=1, padding=1)
        self.bn3 = BatchNorm2d(16)
        self.relu3 = ReLU(inplace=True)
        self.maxpool3 = MaxPool2d(kernel_size=2, stride=2)

        self.conv4 = Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.bn4 = BatchNorm2d(32)
        self.relu4 = ReLU(inplace=True)
        self.maxpool4 = MaxPool2d(kernel_size=2, stride=2)

        #linear(input,output)
        self.linear_layers = Linear(32 * 5 * 5, 6)  ## flatten the image here. i.e linear layer!!

    # Defining the forward pass
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)

        # Apply conv2, bn2, relu2 and maxpool2
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)

        # Apply conv3, bn3, relu3 and maxpool3
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)

        # Apply conv4, bn4, relu4 and maxpool4
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.maxpool4(x)

        # Flatten the output from the conv layers
        x = x.view(x.size(0), -1)

        # Apply the linear layer
        x = self.linear_layers(x)
        return x

In [ ]:
# Initialize the model
model = Net()
model.to(device)  # to GPU/CPU!!!!!!

## sequential method of constructing models

In [ ]:
# sequential way

model = Sequential(
            # Defining a 2D convolution layer
            Conv2d(3, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(4, 8, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(8),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(16),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(32),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            Linear(32 * 5 * 5, 6)
        )

# Visualization of Model

### Visualize through torchsummary

In [ ]:
from torchsummary import summary

In [ ]:
dummy_model = Net().to(device)
summary(dummy_model, (3, 90, 90))  #-1 correspnds to batch size!, -1>> last or all remaining

### Visualize through TorchViz

In [ ]:
!pip install torchviz

In [ ]:
from torchviz import make_dot
dummy_image = next(iter(train_loader))[0]
dummy_model = Net()
y_hat = dummy_model(dummy_image)

In [ ]:
make_dot(y_hat.mean(),params=dict(dummy_model.named_parameters())).render("graph2", format="png")

In [ ]:
dummy_model.state_dict() # show the entire as dict!

### Model Configuration

In [ ]:
# Select a loss function
loss_function = torch.nn.CrossEntropyLoss()

# Select an optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

# Training Loop

Variable Initialization

In [ ]:
len(train_loader) # number of images per batch

In [ ]:
len(train_loader.dataset.indices)

In [ ]:
def run_1_epoch(model, loss_fn, optimizer, loader, train = False):


  if train:
    model.train()
  else:
    model.eval()

  total_correct_preds = 0

  total_samples_in_loader = len(loader.dataset.indices)
  total_batches_in_loader = len(loader)

  total_loss = 0

  for image_batch, labels in tqdm(loader): #TPDM gives you proegress updates!

    # Transfer image_batch to GPU if available
    image_batch = image_batch.to(device) #gives you images of the batch
    labels = labels.to(device) #gives you labels of the batch

    # Zeroing out the gradients for parameters
    if train:
      optimizer.zero_grad() # pytorch doesnt reset gradients to zeros after each iteration and keeps adding to the previous,
      #if this is not done

    # Forward pass on the input batch
    output = model.forward(image_batch)

    # Acquire predicted class indices
    _, predicted = torch.max(output.data, 1) # the dimension 1 corresponds to max along the rows


    # Removing extra last dimension from output tensor
    output.squeeze_(-1)

    # Compute the loss for the minibatch
    loss = loss_function(output, labels)

    # Backpropagation
    if train:
      loss.backward()

    # Update the parameters using the gradients
    if train:
      optimizer.step()

    # Extra variables for calculating loss and accuracy
    # count total predictions for accuracy calcutuon for this epoch
    total_correct_preds += (predicted == labels).sum().item()

    total_loss += loss.item()

  loss = total_loss / total_batches_in_loader
  accuracy = 100 * total_correct_preds / total_samples_in_loader

  return loss, accuracy

In [ ]:
epochs = 30

#train_accuracy_list = []
#val_accuracy_list = []

#train_loss_list = []
#val_loss_list = []

#val_accuracy_max = -1 # used to store best model based on accuracy!

In [ ]:
if torch.cuda.is_available():
    model.cuda()

In [ ]:
# Main training and validation loop for n number of epochs
for i in range(epochs):

  # Train model for one epoch
  print("Epoch %d: Train"%(i))
  train_loss, train_accuracy  = run_1_epoch(model, loss_function, optimizer, train_loader, train= True)

  # Lists for train loss and accuracy for plotting
  train_loss_list.append(train_loss)
  train_accuracy_list.append(train_accuracy)

  # Validate the model on validation set
  print("Epoch %d: Validation"%(i))
  with torch.no_grad():
    val_loss, val_accuracy  = run_1_epoch(model, loss_function, optimizer, val_loader, train= False)

  # Lists for val loss and accuracy for plotting
  val_loss_list.append(val_loss)
  val_accuracy_list.append(val_accuracy)

  print('train loss: %.4f'%(train_loss))
  print('val loss: %.4f'%(val_loss))
  print('train_accuracy %.2f' % (train_accuracy))
  print('val_accuracy %.2f' % (val_accuracy))

  # Save model if validation accuracy for current epoch is greater than
  # all the previous epochs
  if val_accuracy > val_accuracy_max:
    val_accuracy_max = val_accuracy
    print("New Max val Accuracy Acheived %.2f. Saving model.\n\n"%(val_accuracy_max))
    torch.save(model,'best_val_acc_model.pth')
  else:
    print("val accuracy did not increase from %.2f\n\n"%(val_accuracy_max))


**Accuracy and Loss Result Graphs:**

In [ ]:
plt.figure()
plt.plot(train_accuracy_list, label="train_accuracy")
plt.plot(val_accuracy_list, label="val_accuracy")
plt.legend()

plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.title('Training and val Accuracy')

plt.figure()
plt.plot(train_loss_list, label="train_loss")
plt.plot(val_loss_list, label="val_loss")

plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and val Loss')

# Evaluating the Model

**Loading the best saved model:**

In [ ]:
best_val_model1 = torch.load('/content/best_val_acc_model.pth')

In [ ]:
torch.save(best_val_model1,'best_val_acc_model.pth')

In [ ]:
with torch.no_grad():
    test_loss, test_accuracy  = run_1_epoch(model, loss_function, optimizer, test_loader, train= False)

print('test loss: %.4f'%(train_loss))
print('test_accuracy %.2f' % (train_accuracy))


**Scene Sample Inferences:**

In [ ]:
!cp /content/drive/MyDrive/Machvis/Labs/Lab1_pytorch_intro/samples_scenery.zip ./

In [ ]:


#Copying the Pokemon Sample Folder from the drive into the colab disk
!unzip /content/samples_scenery.zip

In [ ]:
#Loop for loading each image and predict the out for each one
#The title of each image is the inference result of the model
pokemon_samples_path = '/content/samples_scenery'
for image_path in os.listdir(pokemon_samples_path):
  try:
    im = Image.open(os.path.join(pokemon_samples_path, image_path))
    im = im.convert("RGB")
    im = im.resize((90,90),Image.ANTIALIAS)
    loader = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
    im_tensor = loader(im)
    im_tensor = im_tensor.view(1,3,90,90)
    im_tensor = im_tensor.to(device)  #assumes that you're using GPU
    output = best_val_model1.forward(im_tensor)
    #Get the class-to-index dictionary
    class_to_index_dict = train_loader.sampler.data_source.dataset.class_to_idx
    #Get the predicted class label
    prediction_label = output.argmax()
    plt.figure()
    #Print the title corresponding to the predicted label
    plt.title([k for k,v in class_to_index_dict.items() if v == prediction_label],fontweight="bold")
    plt.imshow(im)
  except IOError:
    pass

# Credits
Notebook Prepared by Bostan Khan, Team Lead, MachVIS Lab